In [3]:
%matplotlib inline
import segpy
from segpy.reader import create_reader
from segpy.writer import write_segy

import numpy as np
import matplotlib
import matplotlib.pyplot
from PIL import Image

import os
import shutil

from collections import defaultdict, Counter
import joblib
import pickle

from scipy.stats import mode
from scipy.special import kl_div
from scipy.stats import entropy

from sklearn.preprocessing import MinMaxScaler

%run bp_smc_example.ipynb

In [2]:
len(os.listdir('small_dataset'))

120

In [3]:
# split up stack and gather files into separate directories
for filename in os.listdir('small_dataset'):
    src = 'small_dataset/' + filename
    if 'gather' in filename:
        dst = 'gather/' + filename
        shutil.copy(src, dst)
    if 'stack' in filename:
        dst = 'stack/' + filename
        shutil.copy(src, dst)

In [5]:
print_segy_info('stack/img_12.stack.segy')

Filename:              stack/img_12.stack.segy
SEG Y revision:        SegYRevision.REVISION_0
Number of traces:      1058
Data format:           IBM 32 bit float
Dimensionality:        0

=== BEGIN TEXTUAL REEL HEADER ===
C01  BP syntheitc Date: 05/20/2019                                              
C02  image stack scenario with the following dimension                          
C03  axis  :         z         x         y                                      
C04  size  :       400      1058         1                                      
C05  origin:   0.00000   0.00000   0.00000                                      
C06  delta :  10.00000  10.00000  10.00000                                      
=== END TEXTUAL REEL HEADER ===


In [19]:
# make a dictionary of arrays for every realization in stack
stack_dict = {}
for filename in os.listdir('stack')[1:-1]:
    arr = np.zeros((1058, 400))

    with open(f'stack/{filename}', 'rb') as segy_in_file:
        segy_reader = create_reader(segy_in_file, endian='>')

        for trace_index in segy_reader.trace_indexes():
            data = segy_reader.trace_samples(trace_index)
            for i in range(len(data)):
                arr[trace_index, i] = data[i]
        stack_dict[filename] = arr

In [20]:
# make a dictionary of arrays for every realization in stack
gather_dict = {}
for filename in os.listdir('gather')[1:-1]:
    arr = np.zeros((1058, 39, 400))
    with open(f'gather/{filename}', 'rb') as segy_in_file:
        segy_reader = create_reader(segy_in_file, endian='>')

        count = 0
        for trace_index in segy_reader.trace_indexes():
            data = segy_reader.trace_samples(trace_index)
            for i in range(len(data)):
                arr[int(trace_index / 39), trace_index % 39, i] = data[i]
        gather_dict[filename] = arr

In [6]:
# load the two dictionaries into the directory to use later
# joblib.dump(stack_dict, 'stack_dict')
stack_dict = joblib.load(open('stack_dict', 'rb'))

# joblib.dump(gather_dict, 'gather_dict')
gather_dict = joblib.load(open('gather_dict', 'rb'))

In [7]:
thing = stack_dict['img_12.stack.segy']
print(len(thing[0]))

print(len(thing))

# there 1058 instances of 400 elements arrays

400
1058


In [8]:
# turn dictionary into pixel arrays (1058 x 400)
stack_arr = {}

for key in stack_dict.keys():
    arr = stack_dict[key]
    arr = arr.transpose()
    min_arr = arr.min()
    max_arr = arr.max()
    arr = (arr - min_arr) / (max_arr - min_arr)
    arr = arr * 255
    stack_arr[key] = arr


In [9]:
def mode_func(x):
    values, counts = np.unique(x, return_counts=True)
    m = counts.argmax()
    return values[m], 

def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

In [116]:
# function computes the uncertainty image of all the stack segy files
cnt = Counter()

def uncertainty_image(pixel_arr):
    kl_scores = []
    kl_arr = np.zeros((58, 1058, 400))
    i = 0
    for key in pixel_arr.keys():
        j = 0
        for arr in pixel_arr[key]:
            bins = np.linspace(arr.min(), arr.max(), num=10)
            binned_arr = np.digitize(arr, bins)
            
            cnt = Counter()
            for val in binned_arr:
                cnt[val] += 1
            cnt = sorted(cnt.items())
            
            freq_arr = [x[1] for x in cnt]
            max_val = max(freq_arr)
            mode_bin = freq_arr.index(max(freq_arr)) + 1
            onehot_arr = [1e-9 if x != max_val else 1 for x in freq_arr]
            freq_arr = np.array(freq_arr) / 400
            

            kl_score = kl_divergence(freq_arr, onehot_arr)
            kl_scores.append(kl_score)
            score_arr = np.full((1, 400), kl_score)
            kl_arr[i, j, :] = score_arr
            j += 1
        score_arr = kl_arr[i, :, :]
        min_arr = arr.min()
        max_arr = arr.max()
        score_arr = (score_arr - min_arr) / (max_arr - min_arr)
        score_arr = score_arr * 255
        im = Image.fromarray(score_arr)
        im = im.convert('RGB')
        images.append(im)
        i += 1
        
#     for i in range(58):
#         images.append(kl_arr[i])
    return kl_scores, images

In [119]:
kl_scores, images = uncertainty_image(stack_arr)
dictionary = sorted(Counter(kl_scores).items())